<center><h1 style="color:#0098E9;"> Topicos avanzados de Ingenieria Matematica</h1></center>
<center><h1 style="color:#0098E9;"> Tarea 2</h1></center>
<center><h3 style="color:#0098E9;"> Manuel A. Sanchez</h3></center>
<center><h3 style="color:#0098E9;"> Pontificia Universidad Catolica de Chile</h3></center>
<hr style="border:2px solid gray"> </hr>

## 5. M\'etodo de HDG para singularidad de esquina en 2d y 3d

Considere la ecuaci\'on de Poisson en un dominion con forma de $L$, es decir considere $\Omega = (-1,1)^{2}\backslash((0,1)\times(-1,0)$

\begin{equation*}
\begin{array}{rclrl}
-\Delta u(x,y)) &=& 0& \mbox{en} & \Omega \\
u &=& u_{D} & \mbox{sobre} & \partial \Omega_{D} =\partial \Omega
\end{array}
\end{equation*}

con soluci\'on $u(x,y) = r^{\alpha} \sin(\alpha\theta)\in H^{1+\alpha-\epsilon}(\Omega)$ para $\alpha=2/3$ y $\epsilon>0$, donde $r = \sqrt{x^{2}+y^{2}}$ y $\theta = \arctan(y/x)$.

- 5.1 Utilice NGsolve para programar el problema utilizando un m\'etodo de HDG con espacios locales $V(K)  =  \mathcal P_{p}$,
$W(K) =  \mathcal P_{p}$, and $ M(\partial K) = \mathcal P_p(\partial K)$ para todo $K\in \mathcal T_{h}$ y par\'ametro de estabilizaci\'on $\tau = 1$, para $p=0,1,2,3$.
- 5.2 Reporte, ya sea en una tabla o un gr\'afico log-log, los errores para las aproximaciones $u_h$, $u_h^*$ y $q_h$  y los respectivos ordenes de convergencia.  Grafique la solucion aproximada.
- 5.3 Observe que los resultados de convergencia son suboptimos debido a la regularidad de la soluci\'on. Investigue posibles soluciones para recuperar la convergencia optima. Muestre sus resultados.
- 5.4 Resuelva el problema en 3 dimensiones sobre $\Omega = (-1,1)^{2}\backslash((0,1)\times(-1,0)\times(0,1)$ 

\begin{equation*}
\begin{array}{rclrl}
-\Delta u(x,y,z)) &=& 1& \mbox{en} & \Omega \\
u &=& 0 & \mbox{sobre} &\partial \Omega
\end{array}
\end{equation*}

Grafique la soluci\'on aproximada.

## Respuesta.

In [12]:
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
from math import pi

In [13]:
from netgen.geom2d import SplineGeometry
def Lshaped_domain():
    geo = SplineGeometry()
    pnts =[(1,0),(1,1),(-1,1),(-1,-1),(0,-1), (0,0)]
    p1,p2,p3,p4,p5,p6 = [geo.AppendPoint(*pnt) for pnt in pnts]
    curves = [[["line",p1,p2],"outer"],
              [["line",p2,p3],"outer"],
              [["line",p3,p4],"outer"],
              [["line",p4,p5],"outer"],
              [["line",p5,p6],"reentrant"],
              [["line",p6,p1],"reentrant"]]
    [geo.Append(c,bc=bc) for c,bc in curves]
    return geo

def Poisson_Lshaped_problem():
    geo = Lshaped_domain()
    alpha      = 2.0/3.0

    r          = sqrt(x**2+y**2)
    
    rinv = 1.0/IfPos(r-1e-15, r, 1.e-15) # threshold to avoid 0-division
    theta = IfPos(y+1e-15, acos(x * rinv), pi - asin(y*rinv))
    uexact     = (r**alpha)*sin(alpha*(theta))
    dxr        = x/r
    dyr        = y/r
    dxtheta    = 1/(1+(y/x)**2)*(-y/x**2)
    dytheta    = 1/(1+(y/x)**2)*(1/x)
    graduexact = CoefficientFunction(( alpha*r**(alpha-1)*dxr*sin(alpha*theta) + r**alpha*cos(alpha*theta)*alpha*dxtheta,\
                                       alpha*r**(alpha-1)*dyr*sin(alpha*theta) + r**alpha*cos(alpha*theta)*alpha*dytheta  ))
    qexact     = -graduexact 
    uD         = uexact
    f          = 0
    return geo, f, uD, uexact, qexact


In [61]:
def SetUp_HDGPoisson(mesh, force, p):
    Vh = VectorL2(mesh, order=p)
    Wh = L2(mesh, order=p)
    Mh = FacetFESpace(mesh, order=p, dirichlet="outer|reentrant")
    fes = FESpace([Vh,Wh,Mh])
    
    qh, uh, uhat = fes.TrialFunction()
    vh, wh, what = fes.TestFunction()

    # stabilization parameter
    tau = 1.0
    # vector normal
    n = specialcf.normal(mesh.dim)
    # HDG numerical flux
    qhatn = qh*n+tau*(uh-uhat)
    
    # bilinear form
    condense=True
    a = BilinearForm(fes, condense=condense)
    a += (qh*vh - uh*div(vh) - qh*grad(wh))*dx
    a += uhat*vh*n*dx(element_boundary=True)
    a += qhatn*wh*dx(element_boundary=True)
    a += qhatn*what*dx(element_boundary=True)

    f = LinearForm(fes)
    f += force*wh*dx
    gf = GridFunction(fes)
    return gf, a, f

def Solve_HDG(mesh, p, force, uD):
    # Solution
    gf, a, f = SetUp_HDGPoisson(mesh, force, p)
    gfq, gfu, gfuhat = gf.components 
    c = Preconditioner(a, "bddc")
    a.Assemble()
    f.Assemble()
    # set Dirichlet boudary condition
    gfuhat.Set(uD, definedon=mesh.Boundaries("outer"))
    # Solve linear systems
    f.vec.data += a.harmonic_extension_trans * f.vec
    solvers.CG(mat = a.mat, pre=c.mat, rhs=f.vec, sol=gf.vec, initialize=False)    
    gf.vec.data += a.harmonic_extension * gf.vec
    gf.vec.data += a.inner_solve * f.vec
    return gf

In [62]:
def Convergence_test(p=1, last_mesh=5, plot=False):
    Lshaped_geo, force, uD, uexact, qexact = Poisson_Lshaped_problem()
    output = []
    for i in range(2,last_mesh):
        h = 2**(1-i)*np.sqrt(2.0)
        mesh = Mesh(Lshaped_geo.GenerateMesh(maxh=h))
        gf = Solve_HDG(mesh, p, force, uD)
        gfq, gfu, gfuhat = gf.components 
        # output
        err_u     = sqrt( Integrate( InnerProduct( gfu - uexact, gfu - uexact), mesh))
        err_q     = sqrt( Integrate( InnerProduct( gfq - qexact, gfq - qexact), mesh))
        output.append ((h, gfu.space.ndof, err_u, err_q))
    # end for
    if plot:
        Draw(gfu)
        Draw(uexact, mesh,'uexact')
    return output

from pandas import DataFrame
def print_markdown_table(errors):
    i = 1
    h = []
    N = []
    error_u = []
    order_u = []
    error_q = []
    order_q = []
    while i < len(errors):
        h.append(errors[i][0])
        N.append(errors[i][1])
        error_u.append(errors[i][2])
        order_u.append(log(errors[i-1][2]/errors[i][2])/log(errors[i-1][0]/errors[i][0]))
        error_q.append(errors[i][3])
        order_q.append(log(errors[i-1][3]/errors[i][3])/log(errors[i-1][0]/errors[i][0]))
        i += 1

    # pretty print
    df = DataFrame({"h": h,"N": N,'$$\|u-u_h\|_{L^2}$$': error_u,"$$\mbox{order } u$$": order_u,"$$\|q-q_h\|_{L^2}$$": error_q, " $$\mbox{order }q$$": order_q})
    display(df)


In [63]:
# Convergence test
output_p0 = Convergence_test( p=0, last_mesh=7);
output_p1 = Convergence_test( p=1, last_mesh=7);
output_p2 = Convergence_test( p=2, last_mesh=7);
output_p3 = Convergence_test( p=3, last_mesh=7);

iteration 0 error = 2.9235860700293084
iteration 1 error = 3.824104709669569e-16
iteration 0 error = 4.791161304313566
iteration 1 error = 1.984175359987624e-15
iteration 0 error = 6.9160164183163335
iteration 1 error = 5.347540782012761e-15
iteration 0 error = 9.806733873643424
iteration 1 error = 1.0671559491217844e-14
iteration 0 error = 13.87749592998795
iteration 1 error = 2.508815639488889e-14
iteration 0 error = 4.058017112776682
iteration 1 error = 0.7096151943131492
iteration 2 error = 0.11052535044728978
iteration 3 error = 0.002840845834153834
iteration 4 error = 7.32167346143068e-15
iteration 0 error = 7.460295935917277
iteration 1 error = 2.664418462574243
iteration 2 error = 0.49162475183685794
iteration 3 error = 0.11882389947931497
iteration 4 error = 0.028876857067949145
iteration 5 error = 0.00654685583277615
iteration 6 error = 0.001784935344148733
iteration 7 error = 0.0002831617600045592
iteration 8 error = 5.850050899741948e-05
iteration 9 error = 1.47852793390804

iteration 0 error = 16.63011775078826
iteration 1 error = 6.778947793644438
iteration 2 error = 2.468889648896925
iteration 3 error = 1.3471723971814447
iteration 4 error = 0.4828172356906436
iteration 5 error = 0.1826952743809494
iteration 6 error = 0.0583559983001882
iteration 7 error = 0.017783953506517878
iteration 8 error = 0.007227185003201509
iteration 9 error = 0.002987689330956602
iteration 10 error = 0.0015852041008971366
iteration 11 error = 0.0005165646739252235
iteration 12 error = 0.00020494154750075138
iteration 13 error = 7.034397280807693e-05
iteration 14 error = 2.7487670617409524e-05
iteration 15 error = 9.359665617454005e-06
iteration 16 error = 3.561867269402502e-06
iteration 17 error = 1.4968718005046046e-06
iteration 18 error = 6.576949571107428e-07
iteration 19 error = 2.575083174242949e-07
iteration 20 error = 8.863540013308388e-08
iteration 21 error = 2.904115232991932e-08
iteration 22 error = 1.1077141366424404e-08
iteration 23 error = 4.404302358171815e-09
i

## 5.2 Convergence Table $p = 0,1,2,3$ from top to bottom

In [64]:
print_markdown_table(output_p0)
print_markdown_table(output_p1)
print_markdown_table(output_p2)
print_markdown_table(output_p3)

,h,N,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,0.353553,54,0.105437,1.539961,0.208356,0.856115
1,0.176777,222,0.049566,1.088972,0.129900,0.681646
2,0.088388,878,0.025619,0.952122,0.089971,0.529870
3,0.044194,3500,0.012516,1.033404,0.054849,0.713999


,h,N,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,0.353553,162,0.006209,2.156651,0.087658,0.944189
1,0.176777,666,0.002030,1.612946,0.053289,0.718042
2,0.088388,2634,0.000942,1.108040,0.036574,0.543010
3,0.044194,10500,0.000298,1.658740,0.021996,0.733560


,h,N,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,0.353553,324,0.001990,2.152716,0.049583,0.958306
1,0.176777,1332,0.000667,1.576158,0.030289,0.711060
2,0.088388,5268,0.000324,1.043244,0.020113,0.590667
3,0.044194,21000,0.000106,1.608127,0.012410,0.696624


,h,N,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,0.353553,540,0.000925,2.048845,0.031867,0.957094
1,0.176777,2220,0.000322,1.520850,0.019811,0.685712
2,0.088388,8780,0.000154,1.063526,0.012797,0.630481
3,0.044194,35000,0.000053,1.549407,0.008048,0.669120


In [39]:
def AMR_Convergence_test(p=1, plot=False):
    output = []
    # L-shaped geometry
    Lshaped_geo, force, uD, uexact, qexact = Poisson_Lshaped_problem()
    output = []
    # Initial Mesh
    h = 0.5
    mesh = Mesh(Lshaped_geo.GenerateMesh(maxh=h))
    # Solve
    gf = Solve_HDG(mesh, p, force, uD)
    gfq, gfu, gfuhat = gf.components 
    
    while gfu.space.ndof<50000:
    
        # Compute estimator
        Error_estimator(gfq, gfu, mesh)
        # Refine
        mesh.Refine()
        # Solve
        gf = Solve_HDG(mesh, p, force, uD)
        gfq, gfu, gfuhat = gf.components 
        # output
        err_u     = sqrt( Integrate( InnerProduct( gfu - uexact, gfu - uexact), mesh))
        err_q     = sqrt( Integrate( InnerProduct( gfq - qexact, gfq - qexact), mesh))
        output.append (( gfq.space.ndof, gfu.space.ndof, err_u, err_q))
    
    if plot:
        Draw(gfu,mesh)
    return output

def Error_estimator(gfq, gfu, mesh):
    
    err = (gfq+grad(gfu))*(gfq+grad(gfu))
    elerr = Integrate (err, mesh, VOL, element_wise=True)
    maxerr = max(elerr)
    for el in mesh.Elements():
        mesh.SetRefinementFlag(el, elerr[el.nr] > 0.1*maxerr)
    return sqrt(sum(elerr))
from pandas import DataFrame
def print_markdown_table_amr(errors):
    i = 1
    Nu = []
    Nq = []
    error_u = []
    order_u = []
    error_q = []
    order_q = []
    while i < len(errors):
        Nu.append(errors[i][0])
        Nq.append(errors[i][1])
        error_u.append(errors[i][2])
        order_u.append(log(errors[i-1][2]/errors[i][2])/log(sqrt(errors[i][0])/sqrt(errors[i-1][0])))
        error_q.append(errors[i][3])
        order_q.append(log(errors[i-1][3]/errors[i][3])/log(sqrt(errors[i][1])/sqrt(errors[i-1][1])))
        i += 1

    # pretty print
    df = DataFrame({"N_u": Nu,'$$\|u-u_h\|_{L^2}$$': error_u,"$$\mbox{order } u$$": order_u,"N_q":Nq, "$$\|q-q_h\|_{L^2}$$": error_q, " $$\mbox{order }q$$": order_q})
    display(df)

In [41]:
output_amr_p0 = AMR_Convergence_test(p=0, plot=False)
output_amr_p1 = AMR_Convergence_test(p=1, plot=False)
output_amr_p2 = AMR_Convergence_test(p=2, plot=False)
output_amr_p3 = AMR_Convergence_test(p=3, plot=True)

iteration 0 error = 4.039678357657059
iteration 1 error = 1.223094795232478e-15
iteration 0 error = 5.749577296450163
iteration 1 error = 4.832542709581754e-15
iteration 0 error = 8.10775275416179
iteration 1 error = 9.073934601735612e-15
iteration 0 error = 10.426821496848886
iteration 1 error = 2.070659082833978e-14
iteration 0 error = 12.865834731205187
iteration 1 error = 3.6952037673074535e-14
iteration 0 error = 16.174303022801745
iteration 1 error = 6.096624069458068e-14
iteration 0 error = 20.07581488518088
iteration 1 error = 1.6441701844543534e-13
iteration 0 error = 26.00246580512968
iteration 1 error = 3.5670307653529775e-13
iteration 0 error = 6.506625143744865
iteration 1 error = 2.491531983940605
iteration 2 error = 0.29144868402666957
iteration 3 error = 0.07458039720041394
iteration 4 error = 0.00857386764333498
iteration 5 error = 0.0025031980841651704
iteration 6 error = 0.0003739798249241092
iteration 7 error = 0.0001622748880263845
iteration 8 error = 3.08265023993

iteration 0 error = 15.56595970826178
iteration 1 error = 7.755669281673024
iteration 2 error = 2.6879047013042023
iteration 3 error = 1.0950118914194642
iteration 4 error = 0.3279757717643844
iteration 5 error = 0.08359804187753862
iteration 6 error = 0.02830129062643077
iteration 7 error = 0.008855493085060034
iteration 8 error = 0.0040409352840252545
iteration 9 error = 0.001682495027090468
iteration 10 error = 0.000584528455823902
iteration 11 error = 0.00023581209331196602
iteration 12 error = 9.090282476387394e-05
iteration 13 error = 3.074599998258977e-05
iteration 14 error = 1.3031013476042847e-05
iteration 15 error = 4.693956076865595e-06
iteration 16 error = 1.7373247776591747e-06
iteration 17 error = 6.79015397365138e-07
iteration 18 error = 2.9314082004723453e-07
iteration 19 error = 1.0642830577809079e-07
iteration 20 error = 4.329433910256783e-08
iteration 21 error = 1.5277270082055548e-08
iteration 22 error = 5.937386472596918e-09
iteration 23 error = 2.0809901662850868e

iteration 0 error = 8.54779318479864
iteration 1 error = 4.481021929674899
iteration 2 error = 2.0060855186867252
iteration 3 error = 0.7304670768464281
iteration 4 error = 0.35156750853787727
iteration 5 error = 0.11679967327166853
iteration 6 error = 0.04169243298786959
iteration 7 error = 0.015516074052632617
iteration 8 error = 0.006194813806928234
iteration 9 error = 0.002518415650146201
iteration 10 error = 0.0011569915157724452
iteration 11 error = 0.0005620593303912072
iteration 12 error = 0.00022815649966377146
iteration 13 error = 0.00010948146691895626
iteration 14 error = 4.306012878352195e-05
iteration 15 error = 1.792056321903637e-05
iteration 16 error = 8.316039013312471e-06
iteration 17 error = 3.05577460332952e-06
iteration 18 error = 1.3183555525696513e-06
iteration 19 error = 5.762769869928904e-07
iteration 20 error = 2.3764176102903013e-07
iteration 21 error = 9.827523890686384e-08
iteration 22 error = 4.4767120324702356e-08
iteration 23 error = 1.6930197733550454e-

iteration 0 error = 13.358364989105882
iteration 1 error = 6.524844010278459
iteration 2 error = 4.016840409075727
iteration 3 error = 1.0773474892566697
iteration 4 error = 0.45823584766634673
iteration 5 error = 0.1980746695374352
iteration 6 error = 0.07356005344031327
iteration 7 error = 0.02921863972311846
iteration 8 error = 0.01228105103612911
iteration 9 error = 0.005281200513267003
iteration 10 error = 0.002397029661217544
iteration 11 error = 0.0011575992853193975
iteration 12 error = 0.0004944432568878697
iteration 13 error = 0.00022870201112873775
iteration 14 error = 9.20874793728798e-05
iteration 15 error = 4.258871103501812e-05
iteration 16 error = 1.8232946151844424e-05
iteration 17 error = 8.633295353181028e-06
iteration 18 error = 4.107880734052206e-06
iteration 19 error = 1.6437128810300824e-06
iteration 20 error = 7.081890024798596e-07
iteration 21 error = 3.272465454318072e-07
iteration 22 error = 1.5298079831705141e-07
iteration 23 error = 6.430275027059984e-08
it

iteration 0 error = 21.919173804557275
iteration 1 error = 11.997143905221398
iteration 2 error = 6.411855197066851
iteration 3 error = 2.7140891604867234
iteration 4 error = 0.7601397043523265
iteration 5 error = 0.31045906504156595
iteration 6 error = 0.12615736285038903
iteration 7 error = 0.047763042537857577
iteration 8 error = 0.021831183792646338
iteration 9 error = 0.01018040072396152
iteration 10 error = 0.00469978597443399
iteration 11 error = 0.0019564768789091763
iteration 12 error = 0.0008910754482490076
iteration 13 error = 0.00036470938802149846
iteration 14 error = 0.00018000965873089732
iteration 15 error = 9.210867745398122e-05
iteration 16 error = 4.274566280767729e-05
iteration 17 error = 1.6891301520979362e-05
iteration 18 error = 7.759974001931087e-06
iteration 19 error = 3.2466880392447393e-06
iteration 20 error = 1.4942331702778406e-06
iteration 21 error = 6.460050291884879e-07
iteration 22 error = 3.3036005490688645e-07
iteration 23 error = 1.5144649589181488e-

iteration 0 error = 9.960815029072723
iteration 1 error = 5.654862938411147
iteration 2 error = 3.2126451360066812
iteration 3 error = 1.5251165408691407
iteration 4 error = 0.7631948395440243
iteration 5 error = 0.2729726150139408
iteration 6 error = 0.09154028179588583
iteration 7 error = 0.03550212330979501
iteration 8 error = 0.01979270301451402
iteration 9 error = 0.011103750896554255
iteration 10 error = 0.005919172192350533
iteration 11 error = 0.002656879757496522
iteration 12 error = 0.001302780839434077
iteration 13 error = 0.0006139910726627298
iteration 14 error = 0.00035757061922045923
iteration 15 error = 0.00016630977460600042
iteration 16 error = 7.78756732434335e-05
iteration 17 error = 3.728593655085385e-05
iteration 18 error = 1.738357106383787e-05
iteration 19 error = 8.547170591920083e-06
iteration 20 error = 4.439259240507238e-06
iteration 21 error = 1.9412977026992366e-06
iteration 22 error = 1.0813159002087826e-06
iteration 23 error = 4.6559684539781774e-07
iter

iteration 0 error = 10.706349221128193
iteration 1 error = 6.350841201981397
iteration 2 error = 3.788358192278995
iteration 3 error = 1.5454073331382177
iteration 4 error = 0.7546374033400315
iteration 5 error = 0.3030207755131808
iteration 6 error = 0.11905508351995113
iteration 7 error = 0.04475550310770153
iteration 8 error = 0.025939189984204743
iteration 9 error = 0.014117775057712712
iteration 10 error = 0.006788248036554571
iteration 11 error = 0.002889659165878292
iteration 12 error = 0.0015707163584621252
iteration 13 error = 0.0007653077234677578
iteration 14 error = 0.0003680006932897258
iteration 15 error = 0.00015084520184801664
iteration 16 error = 7.172526150813333e-05
iteration 17 error = 3.946859011421704e-05
iteration 18 error = 1.6876120450501213e-05
iteration 19 error = 9.317587188443887e-06
iteration 20 error = 4.791813403836473e-06
iteration 21 error = 2.3723727255261366e-06
iteration 22 error = 1.1090623620258563e-06
iteration 23 error = 5.329864365109451e-07
it

iteration 0 error = 14.735167606522166
iteration 1 error = 8.565267655003451
iteration 2 error = 6.454789973819493
iteration 3 error = 2.2256454725942882
iteration 4 error = 1.1607139411351715
iteration 5 error = 0.486539792056394
iteration 6 error = 0.21121588864487628
iteration 7 error = 0.09303614585475753
iteration 8 error = 0.05269488976814361
iteration 9 error = 0.027966239571235726
iteration 10 error = 0.014092851555442109
iteration 11 error = 0.0075292779472959865
iteration 12 error = 0.003501250936055582
iteration 13 error = 0.001631387694162266
iteration 14 error = 0.0008642861167218497
iteration 15 error = 0.0004209882307873657
iteration 16 error = 0.00019431210914827473
iteration 17 error = 0.00011022630758417217
iteration 18 error = 5.9164795503372984e-05
iteration 19 error = 3.159884112155101e-05
iteration 20 error = 1.7656244975548726e-05
iteration 21 error = 8.567895655046334e-06
iteration 22 error = 4.437721575186463e-06
iteration 23 error = 2.074375434969363e-06
itera

iteration 0 error = 20.212442930460323
iteration 1 error = 11.22831622945816
iteration 2 error = 8.866558251824708
iteration 3 error = 3.981163498601091
iteration 4 error = 1.1890790612138562
iteration 5 error = 0.5521480023971792
iteration 6 error = 0.22780045098942583
iteration 7 error = 0.12693304567573582
iteration 8 error = 0.06680107823817916
iteration 9 error = 0.03544754472926494
iteration 10 error = 0.020020761498246324
iteration 11 error = 0.01119039632390222
iteration 12 error = 0.005307588412485186
iteration 13 error = 0.002411010852366868
iteration 14 error = 0.0010530761632851781
iteration 15 error = 0.0005315731473705585
iteration 16 error = 0.0002776282108534309
iteration 17 error = 0.0001528461647387079
iteration 18 error = 6.929068421188899e-05
iteration 19 error = 3.989193718761255e-05
iteration 20 error = 1.992957573207928e-05
iteration 21 error = 1.1397981576862036e-05
iteration 22 error = 5.688785173017168e-06
iteration 23 error = 2.873108398130379e-06
iteration 2

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

## 5.3 Convergente test with adaptive mesh refinement, $p=0,1,2,3$

In [42]:
print_markdown_table_amr(output_amr_p0)
print_markdown_table_amr(output_amr_p1)
print_markdown_table_amr(output_amr_p2)
print_markdown_table_amr(output_amr_p3)

,N_u,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,N_q,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,768,0.039826,1.001313,384,0.119669,0.648094
1,2920,0.020276,1.010961,1460,0.075949,0.680857
2,7220,0.013494,0.899575,3610,0.048856,0.974703
3,16956,0.008724,1.021781,8478,0.031715,1.012187
4,48672,0.005057,1.034054,24336,0.020263,0.849723
5,116088,0.003378,0.928250,58044,0.013025,1.016744


,N_u,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,N_q,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,636,0.002675,2.762999,318,0.047260,2.004414
1,864,0.002218,1.221997,432,0.030687,2.818973
2,1692,0.001089,2.116680,846,0.019377,1.368127
3,2568,0.000687,2.208304,1284,0.012392,2.143087
4,4176,0.000465,1.608909,2088,0.007909,1.846801
5,6624,0.000286,2.101790,3312,0.005057,1.939105
6,10140,0.000182,2.134897,5070,0.003239,2.093121
7,16212,0.000121,1.715500,8106,0.002068,1.911211
8,25992,0.000073,2.179638,12996,0.001321,1.900616
9,40752,0.000045,2.102705,20376,0.000843,1.994076


,N_u,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,N_q,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,1152,5.817972e-04,3.818039,576,0.025372,1.941873
1,1584,3.081736e-04,3.990902,792,0.016057,2.873306
2,2016,2.433537e-04,1.958410,1008,0.010182,3.777284
3,2592,1.280479e-04,5.110029,1296,0.006444,3.641453
4,3024,1.248097e-04,0.332318,1512,0.004139,5.743717
5,4224,6.939124e-05,3.513023,2112,0.002619,2.738376
6,5328,5.658209e-05,1.757743,2664,0.001672,3.866258
7,7152,2.711369e-05,4.997375,3576,0.001059,3.103030
8,8880,2.077557e-05,2.460713,4440,0.000680,4.096546
9,12840,1.180436e-05,3.065968,6420,0.000429,2.491981


,N_u,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,N_q,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$
0,1920,2.477919e-04,3.003957,960,0.016344,1.492483
1,2640,9.930330e-05,5.742816,1320,0.010320,2.887650
2,3360,4.174511e-05,7.186840,1680,0.006510,3.821527
3,4080,2.157391e-05,6.799661,2040,0.004105,4.749323
4,4800,1.626417e-05,3.476766,2400,0.002590,5.667816
5,5520,1.526139e-05,0.910669,2760,0.001637,6.565239
6,6240,1.509745e-05,0.176182,3120,0.001040,7.407792
7,7400,6.288343e-06,10.273656,3700,0.000655,5.422567
8,8520,5.760911e-06,1.243143,4260,0.000418,6.364077
9,11000,2.664192e-06,6.037246,5500,0.000262,3.648115


## 3d problem

In [43]:
from netgen.csg import *
#import netgen.gui

geo = CSGeometry()
left  = Plane (Pnt(0,0.5,0.5), Vec(-1,0,0) ).bc('outer')
right = Plane (Pnt(1,0.5,0.5), Vec( 1,0,0) ).bc('outer')
front = Plane (Pnt(0.5,0,0.5), Vec(0,-1,0) ).bc('outer')
back  = Plane (Pnt(0.5,1,0.5), Vec(0, 1,0) ).bc('outer')
bot   = Plane (Pnt(0.5,0.5,0), Vec(0,0,-1) ).bc('outer')
top   = Plane (Pnt(0.5,0.5,1), Vec(0,0, 1) ).bc('outer')

mid1  = Plane (Pnt(0.5,0.5,0.5), Vec(0,0,-1)).bc('outer')
mid2  = Plane (Pnt(1,0.5,1), Vec(0,-1,0)).bc('outer')
mid3  = Plane (Pnt(0.5,1,1), Vec(-1,0,0)).bc('outer')

cube = left * right * front * back * bot * top 
Lshaped =cube -  mid1 * mid2 * mid3
#geo.Add (cube)
geo.Add (Lshaped)
#geo.Draw()

0

## 5.4 3d L-shaped domian 

In [52]:
# mesh
mesh = Mesh(geo.GenerateMesh(maxh=0.25))
# Problem's data
uD = 0
force = 1
# polynomial order
p=3
# Solver
gf = Solve_HDG(mesh, p, force, uD)

iteration 0 error = 0.11686536380278491
iteration 1 error = 0.11246136950260563
iteration 2 error = 0.049953782517455275
iteration 3 error = 0.02630103805466375
iteration 4 error = 0.013652410253343121
iteration 5 error = 0.00647239516992453
iteration 6 error = 0.0033036113979033645
iteration 7 error = 0.0018906741006245618
iteration 8 error = 0.0011121859853699366
iteration 9 error = 0.0005673345254190098
iteration 10 error = 0.0002980257208002642
iteration 11 error = 0.00016483195493806517
iteration 12 error = 9.063593312306272e-05
iteration 13 error = 5.1098201426994175e-05
iteration 14 error = 2.7720022151651518e-05
iteration 15 error = 1.5613217830647476e-05
iteration 16 error = 8.68335630971788e-06
iteration 17 error = 5.13183724339727e-06
iteration 18 error = 2.852760588143554e-06
iteration 19 error = 1.6714672090476254e-06
iteration 20 error = 9.008221234943705e-07
iteration 21 error = 4.926919728828999e-07
iteration 22 error = 2.7381552959219274e-07
iteration 23 error = 1.5031

In [53]:
gfq, gfu, gfuhat = gf.components 
Draw(gfu)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': T…